_Note: This is a SQL scoped notebook, and therefore should be attached to a Serverless SQL warehouse for execution._  

In [0]:
DECLARE OR REPLACE VARIABLE catalog_use STRING DEFAULT 'fhir_workshop';
DECLARE OR REPLACE VARIABLE schema_use STRING DEFAULT REPLACE(SPLIT(current_user(), '@')[0], '.', '_');

In [0]:
SET VARIABLE catalog_use = :catalog_use; 
SET VARIABLE schema_use = :schema_use;

In [0]:
USE IDENTIFIER(catalog_use || "." || schema_use);

In [0]:
SELECT current_catalog(), current_schema();

In [0]:
CREATE OR REFRESH STREAMING TABLE bundle_meta_exploded 
COMMENT 'Original FHIR Bundle Metadata'
TBLPROPERTIES (
  'delta.enableChangeDataFeed' = 'true'
  ,'delta.enableDeletionVectors' = 'true'
  ,'delta.enableRowTracking' = 'true'
  ,'quality' = 'bronze'
  ,'pipelines.channel' = 'PREVIEW'
  ,'delta.feature.variantType-preview' = 'supported'
)
AS SELECT 
  bundle_uuid
  ,file_metadata
  ,ingest_time
  ,fhir:resourceType::string as resourceType
  ,fhir:type::string as type
  ,meta.*
FROM 
  STREAM (fhir_bronze_variant)
  ,LATERAL variant_explode(fhir:Meta) as meta
;

In [0]:
DECLARE OR REPLACE VARIABLE meta_keys STRING;

SET VAR meta_keys = (
  SELECT 
    array_join(collect_list(DISTINCT key), "', '")
  FROM 
    bundle_meta_exploded
);

select meta_keys;

In [0]:
DECLARE OR REPLACE VARIABLE meta_stmnt STRING;

SET VARIABLE meta_stmnt = "
CREATE OR REFRESH STREAMING TABLE Meta
COMMENT 'Parsed FHIR Meta Records'
TBLPROPERTIES (
  'delta.enableChangeDataFeed' = 'true'
  ,'delta.enableDeletionVectors' = 'true'
  ,'delta.enableRowTracking' = 'true'
  ,'quality' = 'bronze'
  ,'pipelines.channel' = 'PREVIEW'
  ,'delta.feature.variantType-preview' = 'supported'
)
AS SELECT
  *
FROM (
  SELECT
    bundle_uuid
    ,file_metadata
    ,ingest_time
    ,resourceType
    ,type
    ,key
    ,value
  FROM 
    STREAM(bundle_meta_exploded)
  )
  PIVOT (
    first(value) FOR key IN ('" || meta_keys || "')
  );
";

SELECT meta_stmnt;

In [0]:
EXECUTE IMMEDIATE meta_stmnt;

In [0]:
SELECT * FROM Meta ORDER BY ingest_time DESC LIMIT 100;